In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
from google.colab import files

uploaded = files.upload()


import pandas as pd
cars = pd.read_csv('EV_cars.csv')
cars.head()

Saving EV_cars.csv to EV_cars (1).csv


,Battery,Car_name,Car_name_link,Efficiency,Fast_charge,Price.DE.,Range,Top_speed,acceleration..0.100.
0,75.0,Tesla Model Y Long Range Dual Motor,https://ev-database.org/car/1619/Tesla-Model-Y...,172,670.0,59017.0,435,217,5.0
1,57.5,Tesla Model 3,https://ev-database.org/car/1991/Tesla-Model-3,137,700.0,46220.0,420,201,6.1
2,60.5,BYD ATTO 3,https://ev-database.org/car/1782/BYD-ATTO-3,183,370.0,44625.0,330,160,7.3
3,61.7,MG MG4 Electric 64 kWh,https://ev-database.org/car/1708/MG-MG4-Electr...,171,630.0,39990.0,360,160,7.9
4,75.0,Tesla Model 3 Long Range Dual Motor,https://ev-database.org/car/1992/Tesla-Model-3...,149,780.0,55220.0,505,201,4.4


In [7]:
cars.isnull().sum()

,0
Battery,0
Car_name,0
Car_name_link,0
Efficiency,0
Fast_charge,2
Price.DE.,51
Range,0
Top_speed,0
acceleration..0.100.,0


In [8]:
# Fill null with mean of each attributes
cars['Fast_charge'] = cars['Fast_charge'].fillna(cars['Fast_charge'].mean())
cars['Price.DE.'] = cars['Price.DE.'].fillna(cars['Price.DE.'].mean())

In [9]:
# Renaming Some Columns
cars.rename(columns={
    "Price.DE." : "Price",
    "acceleration..0.100." : "acceleration_time"}, inplace=True
)
cars = cars.drop('Car_name_link', axis=1)
cars

,Battery,Car_name,Efficiency,Fast_charge,Price,Range,Top_speed,acceleration_time
0,75.0,Tesla Model Y Long Range Dual Motor,172,670.0,59017.0,435,217,5.0
1,57.5,Tesla Model 3,137,700.0,46220.0,420,201,6.1
2,60.5,BYD ATTO 3,183,370.0,44625.0,330,160,7.3
3,61.7,MG MG4 Electric 64 kWh,171,630.0,39990.0,360,160,7.9
4,75.0,Tesla Model 3 Long Range Dual Motor,149,780.0,55220.0,505,201,4.4
...,...,...,...,...,...,...,...,...
355,46.3,Citroen e-Jumpy Combi XL 50 kWh,257,290.0,52730.0,180,130,12.1
356,46.3,Fiat E-Ulysse L2 50 kWh,250,290.0,55990.0,185,130,12.1
357,46.3,Opel Vivaro-e Combi L 50 kWh,257,290.0,51825.0,180,130,12.1
358,68.0,Opel Zafira-e Life M 75 kWh,257,290.0,69250.0,265,130,13.3


In [11]:
# Check Outlier on Numeric Columns
def outlier_thresholds(dataframe, col_name, q1=0.05, q3=0.95):
    quartile1 = dataframe[col_name].quantile(q1)
    quartile3 = dataframe[col_name].quantile(q3)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

def check_outlier(dataframe, col_name):
    low_limit, up_limit = outlier_thresholds(dataframe, col_name)
    if dataframe[(dataframe[col_name] > up_limit) | (dataframe[col_name] < low_limit)].any(axis=None):
        return True
    else:
        return False

In [13]:
num_cols = cars.drop("Car_name", axis=1)
num_cols

,Battery,Efficiency,Fast_charge,Price,Range,Top_speed,acceleration_time
0,75.0,172,670.0,59017.0,435,217,5.0
1,57.5,137,700.0,46220.0,420,201,6.1
2,60.5,183,370.0,44625.0,330,160,7.3
3,61.7,171,630.0,39990.0,360,160,7.9
4,75.0,149,780.0,55220.0,505,201,4.4
...,...,...,...,...,...,...,...
355,46.3,257,290.0,52730.0,180,130,12.1
356,46.3,250,290.0,55990.0,185,130,12.1
357,46.3,257,290.0,51825.0,180,130,12.1
358,68.0,257,290.0,69250.0,265,130,13.3


In [14]:
for col in num_cols:
    print(col, check_outlier(cars, col))

Battery False
Efficiency False
Fast_charge False
Price False
Range False
Top_speed False
acceleration_time False


In [15]:
X = num_cols

In [16]:
# Create the dataset
x = X.drop(['Price'], axis=1)
y = X['Price']

# Import library
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [17]:
# Train Test Split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Standardize dataset
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [18]:
# Import library
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Create a Model
lr_model = LinearRegression()
lr_model

LinearRegression()

In [19]:
# Fit the trained model
lr_model.fit(x_train, y_train)

LinearRegression()

In [20]:
# Make prediction using the testing set
y_pred = lr_model.predict(x_test)

print('R-squared Score :', r2_score(y_test, y_pred))
print('Root Mean Squared Error :', np.sqrt(mean_squared_error(y_test, y_pred)))

R-squared Score : 0.5655388958773666
Root Mean Squared Error : 22337.765296039248


In [24]:
from sklearn.ensemble import RandomForestRegressor

rf_price_model = RandomForestRegressor()
rf_price_model

RandomForestRegressor()

In [25]:
rf_price_model.fit(x_train, y_train)

RandomForestRegressor()

In [26]:
y_pred2 = rf_price_model.predict(x_test)

print('R-squared Score :', r2_score(y_test, y_pred2))
print('Root Mean Squared Error :', np.sqrt(mean_squared_error(y_test, y_pred2)))

R-squared Score : 0.7620507952980867
Root Mean Squared Error : 16531.27694110761


In [27]:
from sklearn.ensemble import RandomForestRegressor
import joblib

# save with joblib
joblib.dump(rf_price_model, 'rf_price_model.joblib')

['rf_price_model.joblib']

In [30]:
from google.colab import files
files.download('rf_price_model.joblib')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

The following chart specifies the input features and the reference input range.

In [34]:
min_values = x.min()
max_values = x.max()

summary_df = pd.DataFrame({
    'Feature': x.columns,
    'Min': min_values.values,
    'Max': max_values.values
})
summary_df

,Feature,Min,Max
0,Battery,21.3,123.0
1,Efficiency,137.0,295.0
2,Fast_charge,170.0,1290.0
3,Range,135.0,685.0
4,Top_speed,125.0,320.0
5,acceleration_time,2.1,19.1
